<a href="https://colab.research.google.com/github/Leucocitokiller/Proyecto-Fina-NLP/blob/main/Proyecto_final_NLP_Redes_Neuronales_Libenson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importación de Librerías.

In [ ]:
import urllib.request
import numpy as np
import pandas as pd
import os
import time
import sys
#-----librerias para trabajar PLN
!python -m spacy download es_core_news_md
import spacy
import es_core_news_md
#es_core_news_md Medium (modelo mediano):
#Es más pesado y más lento que el sm, pero mucho más preciso. Tiene vectores de palabras, entiende mejor el significado de las palabras.

#-----instalación d librerias para análisis de sentimientos.
!pip install spacy spacy-transformers
!pip install pysentimiento
from pysentimiento import create_analyzer

#----librerias para normalización de textos
import re
from unicodedata import normalize
import unicodedata
from collections import Counter


#----librerias para graficar y wordcloud.
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

#----librerías para trabajar con TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
#----libreria para trabajar con BoW.
from sklearn.feature_extraction.text import CountVectorizer
#----librerias para Machine learning
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score, f1_score

# Procesamiento de la Fuente de Datos.

## Conexión con la fuente de datos.


In [ ]:
# Diccionario con las fuentes y sus URLs
filepath_dict = {
    'yelp': 'https://raw.githubusercontent.com/Leucocitokiller/Proyecto-Fina-NLP/main/yelp_comentarios.csv',
    'amazon': 'https://raw.githubusercontent.com/Leucocitokiller/Proyecto-Fina-NLP/main/amazon_cells_comentarios.csv'

}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['Comentario', 'Valor'], sep=';', encoding='latin-1')
    df['Origen'] = source  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
df.head(1100)

## Normalización de la Fuente de datos.


### Eliminación de signos de puntuación.

In [ ]:
# Definición de función para eliminar los signos de puntuación utilizando re, pero considerando no borrar las vocales con acento.

def remove_punctuation(text):
    # Normaliza el texto a NFKD para separar letras y sus tildes
    text = unicodedata.normalize('NFKD', text)
    # Elimina los caracteres diacríticos (como las tildes)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    # Elimina todo lo que no sea letras, números o espacios
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text


# Aplicar la función a la columna 'review_lower'
df['Comentarios'] = df['Comentario'].apply(remove_punctuation)

In [ ]:
df

### Reducir a minúsculas el texto.

In [ ]:
# Create a new column 'Comentarios_lower' with lowercase values from 'Comentario'
df['Comentarios_lower'] = df['Comentarios'].str.lower()

In [ ]:
df

### Convertir a número la columna Valor para su postprocesamiento.

In [ ]:
# Convertimos la columna rating a valor numérico
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')

In [ ]:
df['Valor']

# NLP

## Pre Procesamiento.

### Generación del objeto de SPacy para utilizar en el procesamiento del texto en español.

In [ ]:
nlp = es_core_news_md.load()

### Convertir texto a minúsculas y Tokenización.

In [ ]:
df['Comentarios_tokenizados'] = df['Comentarios_lower'].apply(lambda text: nlp(text))
df[['Comentarios_lower','Comentarios_tokenizados']].head()

### Remoción de StopWords

In [ ]:
# Lista de palabras que NO queremos eliminar (tienen carga emocional)
palabras_sentimiento = {
    # Positivas
    "bueno", "buena","si","buenísimo", "excelentes", "excelente", "genial", "maravilloso", "maravilla", "fantástico", "fabuloso", "increíble",
    "perfecto", "perfecta", "agradable", "satisfecho", "satisfecha", "contento", "contenta", "encantado", "encantada",
    "amable", "simpático", "simpática", "rápido", "rápida", "cómodo", "cómoda", "eficaz", "eficiente", "fácil",
    "recomendable", "ideal", "espectacular", "feliz", "brillante", "cumplió", "cumple", "funciona", "funciona bien",
    "inmejorable", "confiable", "duradero", "cumplidor", "seguro", "preciso", "elegante", "atento", "responsable",
    "acertado", "destacado", "excepcional", "impecable", "sensacional", "útil", "accesible", "económico", "funcional",
    "intuitivo", "conveniente", "hermoso", "linda", "precioso", "excelente calidad", "vale la pena",

    # Negativas
    "malo","no", "mala", "mal", "pésimo", "pésima","nunca", "horrible", "fatal", "insoportable", "lento", "lenta", "incómodo", "incómoda",
    "decepcionante", "decepcionado", "decepcionada", "sucio", "sucia", "caro", "cara", "inútil", "deficiente", "desagradable",
    "complicado", "problemático", "estafa", "engañado", "engañada", "roto", "rota", "desastroso", "error", "errores",
    "retraso", "tardanza", "frágil", "inestable", "poco fiable", "nunca más", "no volveré", "no recomiendo", "no sirve",
    "no funciona", "arruinado", "falló", "fallando", "demora", "pésima atención", "servicio malo", "mala calidad", "molesto",
    "defecto", "problemas", "fallas", "sin sentido", "basura", "pérdida de dinero", "decepción"
}

# Actualizamos spaCy para que NO considere esas palabras como stopwords
for palabra in palabras_sentimiento:
    lex = nlp.vocab[palabra]
    lex.is_stop = False



def parse_and_remove_stopwords(doc):
    """
    Remueve las stopwords de un objeto spaCy Doc.
    """
    # Filtrar stopwords y obtener los tokens como texto
    tokens_filtrados = [token.text for token in doc if not token.is_stop]
    return tokens_filtrados

# Aplicar la función al DataFrame
df['Comentarios_sin_StopWords'] = df['Comentarios_tokenizados'].apply(parse_and_remove_stopwords)

df[['Comentarios_tokenizados','Comentarios_sin_StopWords']].head()

### Lematizado.

In [ ]:
def lematizar_sin_stopwords(doc):
    """
    Devuelve una lista de lemas excluyendo las stopwords.

    Parámetro:
    - doc: objeto spaCy Doc

    Retorna:
    - Lista de lemas (str) sin stopwords
    """
    return [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

# Aplicar la función y guardar el resultado en una nueva columna
df['Comentarios_lema'] = df['Comentarios_tokenizados'].apply(lematizar_sin_stopwords)

df[['Comentarios_tokenizados','Comentarios_lema']].head(20)

## Procesamiento

### Conteo de Palabras mas comunes.

In [ ]:
def graficar_palabras_comunes(df, origen, top_n=10):
    # Filtrar y aplanar los lemas
    lemas = [lema for lemas in df[df['Origen'] == origen]['Comentarios_lema'] for lema in lemas]
    conteo = Counter(lemas).most_common(top_n)

    # Separar palabras y frecuencias
    palabras, frecuencias = zip(*conteo)

    # Crear gráfico
    plt.figure(figsize=(10, 6))
    plt.barh(palabras, frecuencias, color='skyblue')
    plt.xlabel('Frecuencia')
    plt.title(f'Top {top_n} Palabras Más Comunes - {origen.capitalize()}')
    plt.gca().invert_yaxis()  # Poner la palabra más común arriba
    plt.tight_layout()
    plt.show()

# Graficar para Yelp
graficar_palabras_comunes(df, 'yelp')

# Graficar para Amazon
graficar_palabras_comunes(df, 'amazon')

### Conteo de bigramas más comunes.

In [ ]:
from collections import Counter
from itertools import tee
import matplotlib.pyplot as plt

def generar_bigramas(lista):
    """Devuelve bigramas como tuplas a partir de una lista de palabras"""
    a, b = tee(lista)
    next(b, None)
    return list(zip(a, b))

def graficar_bigramas_comunes(df, origen, top_n=10):
    # Filtrar solo los comentarios del origen y generar bigramas
    bigramas = [
        bigrama
        for lemas in df[df['Origen'] == origen]['Comentarios_lema']
        for bigrama in generar_bigramas(lemas)
    ]

    conteo = Counter(bigramas).most_common(top_n)

    # Convertir tuplas de bigramas a string para graficar
    etiquetas = [' '.join(b) for b, _ in conteo]
    frecuencias = [f for _, f in conteo]

    # Crear gráfico
    plt.figure(figsize=(10, 6))
    plt.barh(etiquetas, frecuencias, color='mediumseagreen')
    plt.xlabel('Frecuencia')
    plt.title(f'Top {top_n} Bigramas Más Comunes - {origen.capitalize()}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# Ejemplo de uso
graficar_bigramas_comunes(df, 'yelp')
graficar_bigramas_comunes(df, 'amazon')


### WordClouds

#### WordCloud Yelp.

In [ ]:
# Filtrar el DataFrame
df_yelp = df[df['Origen'] == 'yelp']

# Unir todos los lemas en un solo string (comentarios lematizados ya están en listas)
texto_yelp = ' '.join([' '.join(lemas) for lemas in df_yelp['Comentarios_lema']])

# Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto_yelp)

# Mostrar la nube
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Nube de Palabras - Comentarios de Yelp")
plt.show()

#### WordCloud Amazon.

In [ ]:
# Filtrar el DataFrame
df_amazon = df[df['Origen'] == 'amazon']

# Unir todos los lemas en un solo string (comentarios lematizados ya están en listas)
texto_amazon = ' '.join([' '.join(lemas) for lemas in df_amazon['Comentarios_lema']])

# Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto_amazon)

# Mostrar la nube
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Nube de Palabras - Comentarios de Yelp")
plt.show()


#### WordCloud + Bigramas.

In [ ]:
def generar_bigramas_spacy(df, origen, top_n=50):
    """
    Genera bigramas usando spaCy a partir de la columna 'Comentarios_Lema', sin stopwords.
    Luego genera una nube de palabras.
    """
    # Filtrar los comentarios por 'origen' (por ejemplo, 'yelp' o 'amazon')
    comentarios = df[df['Origen'] == origen]['Comentarios_sin_StopWords']

    # Generar bigramas
    bigramas = []
    for comentario in comentarios:
        # Crear un Doc de spaCy a partir de la lista de lemas (de la columna 'Comentarios_Lema')
        doc = nlp(' '.join(comentario))  # Unimos la lista de lemas y lo procesamos con spaCy
        # Extraer bigramas
        for i in range(len(doc) - 1):
            if not doc[i].is_stop and not doc[i+1].is_stop:  # Asegurarse de que no sean stopwords
                bigramas.append((doc[i].lemma_, doc[i+1].lemma_))

    # Contar los bigramas más comunes
    conteo_bigramas = Counter(bigramas).most_common(top_n)

    # Convertir los bigramas a formato texto "palabra1 palabra2"
    bigramas_texto = {' '.join(bigrama): freq for bigrama, freq in conteo_bigramas}

    # Generar la nube de palabras de los bigramas
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(bigramas_texto)

    # Mostrar la nube
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Word Cloud de Bigramas - {origen.capitalize()}")
    plt.show()

# Generar la nube de bigramas para Yelp y Amazon
generar_bigramas_spacy(df, 'yelp')
generar_bigramas_spacy(df, 'amazon')

### 🚀 Análisis de sentimiento en español con pysentimiento

In [ ]:
# Crear analizador de sentimientos
analyzer = create_analyzer(task="sentiment", lang="es")

# Aplicar a una columna de texto
df['Sentimiento'] = df['Comentario'].apply(lambda x: analyzer.predict(x).output)
# Sentimiento solo guarda lo predicho (POS, NEU o NEG)

df['Probabilidad'] = df['Comentario'].apply(lambda x: analyzer.predict(x).probas)
#Ese diccionario contiene la probabilidad de cada clase: positivo, neutro, negativo Ejemplo: {'POS': 0.84, 'NEU': 0.10, 'NEG': 0.06}.

#### 📊  Gráfico de barras de frecuencia de sentimientos

In [ ]:
sns.countplot(data=df, x='Sentimiento', order=['POS', 'NEU', 'NEG'], palette='pastel')
plt.title('Distribución de Sentimientos totales entre Yelp y Amazon')
plt.xlabel('Sentimiento')
plt.ylabel('Cantidad de Comentarios')
plt.show()


#### Distribución de los sentimientos.

In [ ]:
df['Sentimiento'].value_counts().plot.pie(
    autopct='%1.1f%%',
    startangle=90,
    labels=['Positivo', 'Neutro', 'Negativo'],
    colors=['lightgreen', 'lightblue', 'salmon']
)
plt.title('Distribución porcentual de Sentimientos')
plt.ylabel('')
plt.show()

#### Análisis de Confianza para filtrar comentarios con baja certeza

In [ ]:
# Máxima probabilidad (nivel de certeza del modelo)
df['Confianza'] = df['Probabilidad'].apply(lambda x: max(x.values()))  # En este caso, de la lista {'POS': 0.84, 'NEU': 0.10, 'NEG': 0.06} sólo guarda 0.84 que es el valor mayor

# Filtrar comentarios cuya confianza sea menor a 0.6
comentarios_baja_confianza = df[df['Confianza'] < 0.6]
comentarios_alta_confianza = df[df['Confianza'] >= 0.6]
# Ver los primeros resultados
#comentarios_baja_confianza[['Comentarios', 'Sentimiento', 'Confianza']]

#### Distribución de los comentarios filtrados.

In [ ]:
df[df['Confianza'] >= 0.6]['Sentimiento'].value_counts().plot.pie(
    autopct='%1.1f%%',
    startangle=90,
    labels=['Positivo', 'Neutro', 'Negativo'],
    colors=['lightgreen', 'lightblue', 'salmon']
)
plt.title('Distribución porcentual de Sentimientos con alta confianza')
plt.ylabel('')
plt.show()

#### Distribución de la Confianza de los comentarios.

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(data=df, x='Confianza', bins=20, kde=True, color='skyblue')
plt.axvline(0.6, color='red', linestyle='--', label='Umbral 0.6')
plt.title('Distribución de Confianza del Sentimiento')
plt.xlabel('Confianza')
plt.ylabel('Cantidad de Comentarios')
plt.legend()
plt.show()

# Pruebas de Modelos de Machine Learning.

### División de datos de entrenamiento y prueba.

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    df['Comentario'], df['Valor'], test_size=0.2, random_state=42
)

## Regresión Logística.

### Utilizando TF-IFD.

TF-IDF (Term Frequency - Inverse Document Frequency) es una técnica de procesamiento de texto utilizada para evaluar la importancia de una palabra dentro de un conjunto de documentos. Se basa en dos conceptos:

TF (Frecuencia de Término): Mide cuántas veces aparece un término en un documento específico, comparado con el número total de términos en ese documento. Esto ayuda a capturar cuán relevante es una palabra dentro de un documento en particular.

IDF (Frecuencia Inversa de Documentos): Mide la importancia de una palabra dentro de un conjunto de documentos. Si una palabra aparece en muchos documentos, tiene menos valor. La fórmula es:

Esto ayuda a reducir el peso de las palabras que aparecen frecuentemente en todos los documentos (como "el", "y", "de"), ya que no agregan mucha información.

Así, la importancia de un término en un documento depende tanto de su frecuencia en ese documento como de cuán común es en todo el conjunto de documentos.

#### Cálculo de TF-IDF con TfidVetorizer.

In [ ]:
 # TFIDF espera trabajar con strings y  no listas, por lo que se procede a crear una nueva columna con los datos tokenizados en formato str.
df['Comentarios_sin_StopWords_str'] = df['Comentarios_sin_StopWords'].apply(lambda x: ' '.join(x))

# Crear el vectorizador
tfidfvectorizer = TfidfVectorizer(ngram_range=(1,5))
#inlcuyo bigramas y trigramas para que le de contexto a los comentarios. Esto me permite ver un "No conforme" y no solamente le "No" y el "Conforme" por separado.

# Ajustar y transformar
tfidf_matrix = tfidfvectorizer.fit_transform(df['Comentarios_sin_StopWords_str'])

# Obtener los términos
features = tfidfvectorizer.get_feature_names_out()

# Convertir la matriz a DataFrame
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=features)

# Sumar TF-IDF por columna
tfidf_scores = df_tfidf.sum().sort_values(ascending=False)

# Mostrar top 10
print("🔝 Top 10 n-gramas por score TF-IDF:")
print(tfidf_scores.head(10).round(3))




In [ ]:
# Crear un DataFrame auxiliar con el tipo de n-grama
df_scores = pd.DataFrame({
    'ngram': tfidf_scores.index,
    'score': tfidf_scores.values,
    'tipo': tfidf_scores.index.to_series().apply(lambda x: f'{len(x.split())}-grama')
})

# Ver los 5 más importantes por tipo
top_n = 5
for tipo in ['1-grama', '2-grama', '3-grama']:
    print(f"\n🔝 Top {top_n} {tipo}s:")
    print(df_scores[df_scores['tipo'] == tipo].head(top_n).to_string(index=False))

#### Ajuste de datos de Entrenamiento.

In [ ]:
# Ajustar y transformar los datos de entrenamiento
X_train_tfidf = tfidfvectorizer.fit_transform(X_train)
# Transformar los datos de prueba
X_test_tfidf = tfidfvectorizer.transform(X_test)

#### Generación y prueba de modelo Regresión Logística.

In [ ]:
# Crear un modelo de regresión logística
# Abajo tenés un código con los parámetros expresados de forma que puedas ir modificandolos
model_log_reg = LogisticRegression() #Instanciamos el modelo

# Entrenar el modelo
model_log_reg.fit(X_train_tfidf, y_train) # Fiteamos, es decir, el modelo aprende a partir de los datos de entrenamiento

# Hacer predicciones en el conjunto de prueba
y_pred_log_reg = model_log_reg.predict(X_test_tfidf) # Predecir

#### Evaluación del Modelo.

Matriz de Confusión.

In [ ]:
# 1. Matriz de confusión

# La Matriz de Confusión es útil para Muestra los aciertos y errores del modelo organizados por clase.

cm = confusion_matrix(y_test, y_pred_log_reg)
labels = ['Negativo', 'Positivo']

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión')
plt.show()

Curva ROC AUC.

In [ ]:
# 2. Curva ROC
# ROC AUC SCORE evalúa qué tan bien el modelo separa las clases.
fpr, tpr, _ = roc_curve(y_test, model_log_reg.decision_function(X_test_tfidf))
roc_auc = roc_auc_score(y_test, model_log_reg.decision_function(X_test_tfidf))

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Área bajo la curva ROC AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

Métricas de Predicción.

In [ ]:
# 3. Métricas
# Accuracy:Para medir qué tan bien predice el modelo en datos nuevos (exactitud).
# Accuracy mide el porcentaje total de predicciones correctas sobre el total de casos.
accuracy = accuracy_score(y_test, y_pred_log_reg)
# Precision: Para medir el costo de un falso positivo es alto (por ejemplo, recomendar una película mala como buena).
# Precision mide qué proporción de las predicciones positivas hechas por el modelo son realmente positivas.
precision = precision_score(y_test, y_pred_log_reg)
# Recall: Para medir cuántos de los casos positivos reales fueron capturados por el modelo.
recall = recall_score(y_test, y_pred_log_reg)
# f1 Score: Para medir el promedio armónico entre precisión y recall. Un buen balance si ambas cosas son importantes.
f1 = f1_score(y_test, y_pred_log_reg)

print("Métricas de desempeño del modelo:")
print(f"Accuracy : {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall   : {recall:.2f}")
print(f"F1 Score : {f1:.2f}")

Validación Cruzada.

In [ ]:
# Pipeline que junta vectorizador y modelo
pipeline = make_pipeline(
    TfidfVectorizer(max_features=5000),
    LogisticRegression()
)

# Validación cruzada con 5 particiones (k-fold = 5)
scores = cross_val_score(pipeline, df['Comentario'], df['Valor'], cv=5, scoring='accuracy')

# Resultados
print(f"Precisión media con validación cruzada: {scores.mean():.3f}")
print(f"Desviación estándar: {scores.std():.3f}")

Visualización de palabras asociadas a reseñas positivas y negativas.

In [ ]:
# Obtenemos las palabras del vocabulario
palabras = tfidfvectorizer.get_feature_names_out()

# Coeficientes del modelo (uno por palabra)
coeficientes = model_log_reg.coef_[0]

# Creamos un DataFrame para visualizarlo
df_coef = pd.DataFrame({'palabra': palabras, 'coeficiente': coeficientes})

# Ordenamos por importancia
df_coef = df_coef.sort_values(by='coeficiente', ascending=False)

# En la primera columna veremos el número "índice" de cada palabra según el órden en que fueron procesadas en el modelo.

# Mostramos las 10 palabras más asociadas a valoración positiva y negativa
print("🔼 Palabras más asociadas a reseñas positivas:")
print(df_coef.head(10))

print("\n🔽 Palabras más asociadas a reseñas negativas:")
print(df_coef.tail(10))

Graficar las 10 palabras más positivas y más negativas

In [ ]:
# Colores pastel suaves (más apagados)
color_positivas = '#388e3c'  # verde claro apagado
color_negativas = '#e65100'  # rojo claro apagado

# Top 10 positivas y negativas
top_positivas = df_coef.head(10)
top_negativas = df_coef.tail(10).sort_values(by='coeficiente')

# Crear la figura y los ejes
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico de palabras positivas
bars1 = ax[0].barh(top_positivas['palabra'], top_positivas['coeficiente'], color=color_positivas)
ax[0].set_title('🔼 Palabras asociadas a reseñas positivas', fontsize=14)
ax[0].invert_yaxis()
ax[0].set_xlabel('Coeficiente', fontsize=12)

# Agregar valores al final de las barras (positivas)
for bar in bars1:
    width = bar.get_width()
    ax[0].text(width + 0.01, bar.get_y() + bar.get_height() / 2,
               f'{width:.2f}', va='center', fontsize=10)

# Gráfico de palabras negativas
bars2 = ax[1].barh(top_negativas['palabra'], top_negativas['coeficiente'], color=color_negativas)
ax[1].set_title('🔽 Palabras asociadas a reseñas negativas', fontsize=14)
ax[1].invert_yaxis()
ax[1].set_xlabel('Coeficiente', fontsize=12)

# Agregar valores al final de las barras (negativas)
for bar in bars2:
    width = bar.get_width()
    ax[1].text(width - 0.01, bar.get_y() + bar.get_height() / 2,
               f'{width:.2f}', va='center', ha='right', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# Filtrar bigramas y trigramas
# Cambiar 'ngrama' a 'palabra' para acceder a la columna correcta
df_bi_tri = df_coef[df_coef['palabra'].str.count(' ') >= 2]

top_pos_bi_tri = df_bi_tri.sort_values('coeficiente', ascending=False).head(10)
top_neg_bi_tri = df_bi_tri.sort_values('coeficiente', ascending=True).head(10)

color_positivas = '#388e3c'  # verde oscuro
color_negativas = '#e65100'  # naranja oscuro

fig, axes = plt.subplots(1, 2, figsize=(16,6))

axes[0].barh(top_pos_bi_tri['palabra'], top_pos_bi_tri['coeficiente'], color=color_positivas) # Cambiar 'ngrama' a 'palabra'
axes[0].invert_yaxis()
axes[0].set_title('🔼 Bigrams y Trigrams positivos')
axes[0].set_xlabel('Coeficiente')

axes[1].barh(top_neg_bi_tri['palabra'], top_neg_bi_tri['coeficiente'], color=color_negativas) # Cambiar 'ngrama' a 'palabra'
axes[1].invert_yaxis()
axes[1].set_title('🔽 Bigrams y Trigrams negativos')
axes[1].set_xlabel('Coeficiente')

plt.tight_layout()
plt.show()


🧠 ¿Qué muestran los gráficos?
Las palabras con coeficientes positivos son las que más contribuyen a que el modelo prediga una reseña positiva.

Las palabras con coeficientes negativos son las que más empujan al modelo hacia una predicción negativa.

Prueba del modelo.

In [ ]:
nueva_reseña = "no lo recominedo"  # Reemplaza con la reseña que deseas probar
nueva_reseña_tfidf = tfidfvectorizer.transform([nueva_reseña])
prediccion = model_log_reg.predict(nueva_reseña_tfidf)
# Obtener la probabilidad de la predicción
probabilidadpositiva = model_log_reg.predict_proba(nueva_reseña_tfidf)

# Obtener la probabilidad en la clase predicha (0 o 1)
probabilidad = probabilidadpositiva[0][1]  # Probabilidad de la clase "positivo"

print(f"Se predice que la crítica es de caracter {prediccion[0]}")
print(f" con una probabilidad de que sea positiva de {probabilidad:.2f}")

### Utilizando Bag of Words.

BoW convierte un conjunto de documentos en una matriz de ocurrencias de palabras. A diferencia de TF-IDF, que pondera las palabras según su frecuencia e importancia en relación con todo el corpus, BoW solo cuenta cuántas veces aparece una palabra en un documento sin considerar la frecuencia global de la palabra.

#### Ajuste de datos de entrenamiento para BoW.

In [ ]:
# Instanciamos el vectorizador BoW
vectorizer_bow = CountVectorizer()

# Aplicamos el vectorizador a los comentarios lematizados (ahora en formato string)
vector_bow = vectorizer_bow.fit_transform(df['Comentarios_sin_StopWords_str'])

# Convertimos la matriz de características en un DataFrame para visualizar
bow_df = pd.DataFrame(vector_bow.toarray(), columns=vectorizer_bow.get_feature_names_out())

# Obtener los nombres de las características (palabras)
features_bow = vectorizer_bow.get_feature_names_out()


# Crear un DataFrame con las frecuencias de las palabras
df_bow = pd.DataFrame(vector_bow.toarray(), columns=features_bow)

In [ ]:
# Ajustar y transformar los datos de entrenamiento
X_train_bow = vectorizer_bow.fit_transform(X_train)
# Transformar los datos de prueba
X_test_bow  = X_test_bow  = vectorizer_bow.transform(X_test)

Generación y prueba del Modelo con BoW.

In [ ]:
# Crear un modelo de regresión logística
# Abajo tenés un código con los parámetros expresados de forma que puedas ir modificandolos
model_log_reg_Bow = LogisticRegression() #Instanciamos el modelo

# Entrenar el modelo
model_log_reg_Bow.fit(X_train_bow, y_train) # Fiteamos, es decir, el modelo aprende a partir de los datos de entrenamiento

# Hacer predicciones en el conjunto de prueba
y_pred_log_reg_Bow = model_log_reg_Bow.predict(X_test_bow) # Predecir

#### Evaluación del Modelo.


Matriz de Confusión.

In [ ]:
# 1. Matriz de confusión

# La Matriz de Confusión es útil para Muestra los aciertos y errores del modelo organizados por clase.

cm1 = confusion_matrix(y_test, y_pred_log_reg_Bow)
labels = ['Negativo', 'Positivo']

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusiónc con BoW')
plt.show()

Curva ROC AUC.

In [ ]:
# 2. Curva ROC
# ROC AUC SCORE evalúa qué tan bien el modelo separa las clases.
fpr1, tpr1, _ = roc_curve(y_test, model_log_reg_Bow.decision_function(X_test_bow))
roc_auc1 = roc_auc_score(y_test, model_log_reg_Bow.decision_function(X_test_bow))

plt.figure(figsize=(8, 6))
plt.plot(fpr1, tpr1, color='darkorange', lw=2, label=f'Área bajo la curva ROC AUC = {roc_auc1:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

Métricas de Predicción.

In [ ]:
# 3. Métricas
# Accuracy:Para medir qué tan bien predice el modelo en datos nuevos (exactitud).
# Accuracy mide el porcentaje total de predicciones correctas sobre el total de casos.
accuracy = accuracy_score(y_test, y_pred_log_reg)
# Precision: Para medir el costo de un falso positivo es alto (por ejemplo, recomendar una película mala como buena).
# Precision mide qué proporción de las predicciones positivas hechas por el modelo son realmente positivas.
precision = precision_score(y_test, y_pred_log_reg)
# Recall: Para medir cuántos de los casos positivos reales fueron capturados por el modelo.
recall = recall_score(y_test, y_pred_log_reg)
# f1 Score: Para medir el promedio armónico entre precisión y recall. Un buen balance si ambas cosas son importantes.
f1 = f1_score(y_test, y_pred_log_reg)

print("Métricas de desempeño del modelo:")
print(f"Accuracy : {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall   : {recall:.2f}")
print(f"F1 Score : {f1:.2f}")

Validación Cruzada con BoW.

In [ ]:
# Pipeline con Bag of Words y Regresión Logística
pipeline1 = make_pipeline(
    CountVectorizer(max_features=5000),  # Bag of Words
    LogisticRegression()
)

# Validación cruzada con 5 particiones
scores = cross_val_score(pipeline1, df['Comentario'], df['Valor'], cv=5, scoring='accuracy')

# Resultados
print(f"Precisión media con validación cruzada (BoW): {scores.mean():.3f}")
print(f"Desviación estándar: {scores.std():.3f}")

## Redes Neuronales.